In [1]:
#!pip install ipywidgets
#!pip install requests

import numpy as np
import pandas as pd
from pandas import DataFrame
pd.options.mode.copy_on_write = True 
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from scipy import stats

from geopy.distance import geodesic
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.datasets import make_blobs
from sklearn import datasets, metrics
from active_semi_clustering.semi_supervised.pairwise_constraints import PCKMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

# Pegar data e hora do dia
from datetime import datetime
today = datetime.now()
execucaoInicio = datetime.now()
execucaoFim = datetime.now()
print("-> execução:", execucaoInicio)

# Ajustar path do google drive
#caminhoDrive = "/content/drive/MyDrive/MBA USP TCC/MBA IABG - TCC Orientação/arquivos_dados/"
caminhoDrive = "arquivos/"
#arquivo_consolidado = "dados/output/dados_municipios_consolidado.csv"
arquivo_consolidado = "dados_municipios_consolidado-ajuste_IDHM_PIB-utf8.csv"
arquivo_saida = "tcc_cluster_final_v7r7.csv"
# Carga da fonte de dados consolidada
df = pd.read_csv(caminhoDrive+arquivo_consolidado, sep=';', decimal=",")

# Transformar os dados: experimentando diferentes ajustes nos valores das 3 variáveis selecioadas no trabalho
df['maturidade_minmax'] = minmax_scaler.fit_transform(df['classe_maturidade'].values.reshape(-1,1))
df['IDHM_minmax'] = minmax_scaler.fit_transform(df['classe_IDHM'].values.reshape(-1,1))
df['PIB_minmax'] = minmax_scaler.fit_transform(df['classe_PIB'].values.reshape(-1,1))
df['PIBperCapita_salvo'] = df['PIBperCapita']

# Identificar os outliers da variável PIB per Capita
pib = np.array(df['PIBperCapita'])
zpib = stats.zscore(pib)
df['outlier'] = abs(zpib)
pib_limite = df['PIBperCapita'].max()
pib_minimo = df['PIBperCapita'].min()
pib_median = df['PIBperCapita'].median()
pib_media  = df['PIBperCapita'].mean()
pib_dp  = df['PIBperCapita'].std()
pib_q75  = df['PIBperCapita'].quantile(.75)
pib_q25  = df['PIBperCapita'].quantile(.25)
pib_iq = pib_q75 - pib_q25
pib_lim = pib_q75 + (1.5 * pib_iq)
pib_vlr_zscore1 = pib_media + (1 * pib_dp)
pib_vlr_zscore2 = pib_media + (2 * pib_dp)
pib_vlr_zscore3 = pib_media + (3 * pib_dp)
pib_vlr_zscore1_inf = pib_media - (1 * pib_dp)
pib_vlr_zscore2_inf = pib_media - (2 * pib_dp)
pib_vlr_zscore3_inf = pib_media - (3 * pib_dp)

print(f'- PIB : \n- máximo = {pib_limite:.2f}') 
print(f'- mímimo = {pib_minimo:.2f}') 
print(f'- mediana = {pib_median:.2f}') 
print(f'- média = {pib_media:.2f}')  
print(f'- dp = {pib_dp:.2f}')  
print(f'- q75 = {pib_q75:.2f}')   
print(f'- iq = {pib_iq:.2f}')  
print(f'- limsup = {pib_lim:.2f}')
print(f'- z-score 1 = {pib_vlr_zscore1:.2f}')
print(f'- z-score 2 = {pib_vlr_zscore2:.4f}')
print(f'- z-score 3 = {pib_vlr_zscore3:.4f}')
print(f'- z-score -1 = {pib_vlr_zscore1_inf:.2f}')
print(f'- z-score -2 = {pib_vlr_zscore2_inf:.2f}')
print(f'- z-score -3 = {pib_vlr_zscore3_inf:.2f}')

# Identificar os outliers da variável PIB per Capita
idhm = np.array(df['IDHM'])
zidhm = stats.zscore(idhm)
df['outlier_idhm'] = abs(zidhm)
idhm_limite = df['IDHM'].max()
idhm_minimo = df['IDHM'].min()
idhm_median = df['IDHM'].median()
idhm_media  = df['IDHM'].mean()
idhm_dp  = df['IDHM'].std()
idhm_q75  = df['IDHM'].quantile(.75)
idhm_q25  = df['IDHM'].quantile(.25)
idhm_iq = idhm_q75 - idhm_q25
idhm_lim = idhm_q75 + (1.5 * idhm_iq)
idhm_vlr_zscore2 = idhm_media + (2 * idhm_dp)
idhm_vlr_zscore3 = idhm_media + (3 * idhm_dp)

idhm_vlr_zscore2_inf = idhm_media - (2 * idhm_dp)
idhm_vlr_zscore3_inf = idhm_media - (3 * idhm_dp)


print(f'IDHM:\n- máximo = {idhm_limite:.4f}') 
print(f'- mímimo = {idhm_minimo:.2f}') 
print(f'- mediana = {idhm_median:.4f}') 
print(f'- média = {idhm_media:.4f}')  
print(f'- dp = {idhm_dp:.4f}')  
print(f'- q75 = {idhm_q75:.4f}')   
print(f'- iq = {idhm_iq:.4f}')  
print(f'- limsup = {idhm_lim:.4f}')
print(f'- z-score 2 = {idhm_vlr_zscore2:.4f}')
print(f'- z-score 3 = {idhm_vlr_zscore3:.4f}')
print(f'- z-score -2 = {idhm_vlr_zscore2_inf:.4f}')
print(f'- z-score -3 = {idhm_vlr_zscore3_inf:.4f}')


-> execução: 2024-10-16 14:18:53.668915
- PIB : 
- máximo = 920833.97
- mímimo = 5407.66
- mediana = 23401.17
- média = 33871.28
- dp = 41908.72
- q75 = 40807.45
- iq = 27976.30
- limsup = 82771.90
- z-score 1 = 75780.01
- z-score 2 = 117688.7280
- z-score 3 = 159597.4507
- z-score -1 = -8037.44
- z-score -2 = -49946.16
- z-score -3 = -91854.89
IDHM:
- máximo = 0.8620
- mímimo = 0.00
- mediana = 0.6650
- média = 0.6585
- dp = 0.0746
- q75 = 0.7180
- iq = 0.1190
- limsup = 0.8965
- z-score 2 = 0.8077
- z-score 3 = 0.8823
- z-score -2 = 0.5093
- z-score -3 = 0.4348


In [2]:
# Rotina para processar o agrupamento de cada municipio informado
def Processa_Municipio(ref_cidade=None, df:DataFrame = None, pib_vlr_zscore2:float = 0):
    # # Listar os dados do municipio referência

    if ref_cidade == None:
        print(f"Município referência não informado.")
        return

    ref:DataFrame = df.loc[df['id_municipio'] == ref_cidade]

    if ref.empty:
        print(f"Município {ref_cidade} não encontrado.")
        return
    
    
    if pib_vlr_zscore2 == 0:
        print(f"Valor core 2 do PIB não informado.")
        return

    ref_porte = ref.iloc[0]['codigo_porte']
    ref_pib = ref.iloc[0]['PIBperCapita']
    ref_zscore = ref.iloc[0]['outlier']
    outlier = False
    fator_porte_base  = [(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,10),(7,10),(7,10),(7,10)]    ## ajustar agrupamento de portes conforme as amostras
    fator_porte_amplo = [(1,2),(1,3),(2,4),(3,5),(4,6),(5,7),(5,10),(5,10),(6,10),(6,10)]   ## ajustar agrupamento de portes conforme as amostras (última análise)
#   porte referência       1     2     3     4     5     6     7      8      9      10

    # Avaliar se PIB pesquisado é outlier (considearando o propósito da aplicaçao, utilizando média + 2 desvios padrão)
    if ref_zscore > 2:
        outlier = True
        print("a. id: ", ref_cidade,"- outlier ajustar para fatores amplos de porte e distancia *****")
        fator_porte_base = fator_porte_amplo
        df.loc[df['id_municipio'] == ref_cidade,'PIBperCapita'] = pib_vlr_zscore2
        print("id_municipio:",df.loc[df['id_municipio'] == ref_cidade])
        print("b. id: ", ref_cidade,"- outlier:",outlier, " - ref_pib:", ref_pib, "- pib_vlr_zscore2:", pib_vlr_zscore2)

    df = df.loc[(df['codigo_porte'] >= fator_porte_base[ref_porte-1][0]) & (df['codigo_porte'] <= fator_porte_base[ref_porte-1][1])]     #----> selecionando apenas mesmo porte

    # Verificar se o município pesquisado é Brasília/DF, caso contrário, tirar Brasília da amostra para não ser utilizada como referência na lista final.
    if (ref_cidade != 2605459) & (ref_cidade != 5300108):   	
        df = df.loc[(df['id_municipio'] != 2605459) & (df['id_municipio'] != 5300108)]     #----> selecionando todos, exceto Brasília/DF
        print("id: ", ref_cidade,"- amostra sem Brasilia e Fernando de Noronha:", len(df))
    else:        
        print("id: ", ref_cidade,"- amostra mantendo Brasília e Fernando de Noronha na amostra - cidade pesquisada.")

    # reinicializar indice após a seleção para utilizar nas retrições do PCKMeans
    df.reset_index(inplace=True)

    origem = ref[['Latitude','Longitude']]
    destino = df[['Latitude','Longitude']]
    # Função para calcular distância entre dois pontos geográficos (em metros)
    def calcular_distancia(row1, row2):
        pontoA = (row1['Latitude'], row1['Longitude'])
        pontoB = (row2['Latitude'], row2['Longitude'])
        distancia = geodesic(pontoA, pontoB).km
        return distancia

    # Selecionando os pontos A e B
    pontoA = origem.iloc[0]  # primeira linha'
    
    # Calcular a distância entre a referência e os demais municipios
    for i, pontoB in destino.iterrows():
        df.loc[i, 'distancia'] = calcular_distancia(pontoA, pontoB)

    # Salvar base de dados para preservar processamento
    dfb = df.copy() # salva base selecionada

    # Realizar agrupamento com o Constrained K-means algorithm.
    # Prompt utilizado no Google Colab IA para obter um código python base e, em seguida, realizar os ajustes: 
    # >>> No processamento das restrições must-link (a,b) e cannot-link (a,b), se as restrições CL não forem atendidas, preservar o cluster do primeiro item do par de restrição (a) e realocar o segundo item do par de restrição (b) em outro cluster informado pelo usuário.
    # A entrada X utiliza 3 variáveis (x, y, z).
    # 
    class Constrained_KMeans:
        # Ajustes nos parâmetros do K-means: max_iter=10000, random_state=None, user_cluster_for_relocation=0
        # Ajustes para salvar centroids
        #
        def __init__(self, n_clusters, max_iter=10000, random_state=None, user_cluster_for_relocation=0):
            self.n_clusters = n_clusters
            self.max_iter = max_iter
            self.tol = 1e-4
            self.random_state = random_state
            self.user_cluster_for_relocation = user_cluster_for_relocation
            self.sum_cluster_relocation = 0

        def fit(self, X, ml_constraints=[], cl_constraints=[]):
            kmeans = KMeans(n_clusters=self.n_clusters, init='k-means++', max_iter=self.max_iter, tol=self.tol, random_state=self.random_state)
            kmeans.fit_predict(X)
            labels = kmeans.labels_
            centroids  = kmeans.cluster_centers_       # customização para salvar os centroides

            for _ in range(self.max_iter):
                new_labels = labels.copy()
                for a, b in ml_constraints:
                    if new_labels[a] != new_labels[b]:
                        new_labels[b] = new_labels[a]

                for a, b in cl_constraints:
                    if new_labels[a] == new_labels[b]:
                        new_labels[b] = self.user_cluster_for_relocation     # customização do algoritmo para realocar o b para um cluster informado, evitando a violação
                        self.sum_cluster_relocation += 1                     # contador das realocações

                if np.array_equal(labels, new_labels):
                    break

                labels = new_labels

            self.labels_ = labels
            self.centroid_ = centroids       # customização para salvar os centroides
            return self

        def predict(self, X):
            return self.labels_

    # FASE 1a: Agrupamento sem restrições para identificar os possíveis pares ML/CL
    # Selecionar as 3 variáveis para o agrupamento: classe_maturidade, IDHM, PIB_minmax
    X = df[['classe_maturidade','IDHM','PIBperCapita']].values

    # K-means com 3 variáveis
    n_clusters = 4   #----> escolha após identificar o melhor silhouette_score
    ml = []
    cl = []
    kmeans = Constrained_KMeans(n_clusters=n_clusters, random_state=42, user_cluster_for_relocation=n_clusters+1)
    y = kmeans.fit(X, ml, cl)
    labels = kmeans.labels_
    score = silhouette_score(X, labels)
    score_sample = silhouette_samples(X, labels)
    df['fase1_c_kmeans_score'] = score
    df['fase1_c_kmeans_score_sample'] = score_sample
    df['fase1_c_kmeans_cluster'] = labels

    # Salvar base de dados para preservar processamento
    dfb = df.copy() # salva base selecionada

    # FASE 1b: Construção das Restrições baseada nos grupos do agrupamento anterior
    # Gera lista final de municipios similares em ordem de aderência (maturidade, PIB e IDHM) para identificar cluster referência
    lf = df[['id_municipio','fase1_c_kmeans_cluster','nome_municipio','sigla_uf','codigo_porte','populacao','Latitude','Longitude','distancia', 'nivel_maturidade','IDHM','PIBperCapita','PIBperCapita_salvo',
                    'classe_maturidade','classe_IDHM','classe_PIB','adesaoRedeGOVBR']].sort_values(by=['classe_maturidade','classe_PIB','classe_IDHM'],ascending=False)
    
    # Restaurar base de dados 
    df = dfb.copy()

    # Selecionar 3 variáveis para o agrupamento: classe_maturidade, IDHM, PIB_minmax
    X = df[['classe_maturidade','IDHM','PIBperCapita']].values

    # Salvar referências
    ref_indice     = df[df.id_municipio == ref_cidade].index.tolist()
    ref_maturidade = df[df.id_municipio == ref_cidade]['classe_maturidade'].iloc[0]
    ref_regiao     = df[df.id_municipio == ref_cidade]['codigo_regiao'].iloc[0]
    ref_amostra_selecao = len(df)     # salva total de municipios alocados na amostra

    # Ajustar distancia conforme a região para compensar densidade de cidades (Nota: revisar e ajustar)
    fator_distancia_padrao   = [2000,1200,500,600,600]    # fator de cada região em kilometros geral - NO, NE, SE, SU e CO.
    fator_distancia_especial = [4000,2400,1000,1200,1200]    # fator de cada região em kilometros para municipios com maturidade alta - NO, NE, SE, SU e CO.

    # Forçar fator_distancia_especial quando municipio tem valor outlier, aumentando a amostra da seleção
    if outlier:
        print("c. outlier - distancia especial")
        fator_distancia_padrao = fator_distancia_especial

    # Identificar cluster gerado pela prévia do PCKMeans para montar restrições apenas com os municipios incluidos no mesmo cluster
    ref_cluster    = lf[lf.id_municipio == ref_cidade]['fase1_c_kmeans_cluster'].iloc[0]

    # Retirar municipio referência da lista de restrições
    lf:DataFrame = lf[lf['id_municipio'] != ref_cidade]
    lfs = lf.copy()
    lf = lf[lf['fase1_c_kmeans_cluster'] == ref_cluster]
    ref_amostra_cluster_inicial = len(lf)     # salva total de municipios alocados no cluster referência

    # Construção das Restrições:
    # ML: mínima distância geográfica entre a referência e cada municipio. 
    # Selecionar apenas se não exceder o fator de cada região (kilometros)
    if ref_maturidade < 3:
        # permitir maturidades superiores dentro da distância quando inicial ou intermediário
        lst_ml = lf.loc[(lf['fase1_c_kmeans_cluster'] == ref_cluster) & 
                        (lf['classe_maturidade'] > ref_maturidade) & 
                        (lf['distancia']-fator_distancia_padrao[ref_regiao-1] < 0)].index.tolist()
    else:
        # permitir distancias maiores e maturidades intermediario ou superior quando aprimorado ou avançado (as amostras são menores)
        lst_ml = lf.loc[(lf['fase1_c_kmeans_cluster'] == ref_cluster) & 
                        (lf['classe_maturidade'] >= 2) &
                        (lf['distancia']-fator_distancia_especial[ref_regiao-1] < 0)].index.tolist()

    lst_ml.sort()

    # Preparar os pares ML
    ml=[]
    for i in range(len(lst_ml)):
        tupla=(ref_indice[0], lst_ml[i])
        ml.append(tupla)

    # CL: maturidade inferior à referência e/ou distância superior ao fator de referência.
    # Selecionar todos do cluster de referência que não estão no ML.
    lst_cl = []
    lf = lfs.copy()
    lst_cl = list(lf.loc[~lf.index.isin(lst_ml)].index.tolist())
    lst_cl.sort()

    # Preparar os pares CL
    cl=[]
    for i in range(len(lst_cl)):
        tupla=(ref_indice[0], lst_cl[i])
        cl.append(tupla)

    # FASE 2: Agrupamento com Restrições PCKMeans com pares ML/CL
    executar_Fase2 = False

    if executar_Fase2:
        # Restaurar base com o último processamento
        df = dfb.copy()

        # Selecionando 3 variáveis para o agrupamento: classe_maturidade, IDHM, PIB_minmax
        X = df[['classe_maturidade','IDHM','PIBperCapita']].values
        # Executar PCKMeans  sem aplicar as restrições ML-Must Link e CL-Cannot Link
        pckmeans = PCKMeans(n_clusters=n_clusters, max_iter=40)
        pck = pckmeans.fit(X, cl=[], ml=[])
        labels = pckmeans.labels_
        score = silhouette_score(X, labels)
        score_sample = silhouette_samples(X, labels)
        df['fase2_pckmeans_score'] = score
        df['fase2_pckmeans_score_sample'] = score_sample
        df['fase2_pckmeans_cluster'] = labels
        resultado_ml = [labels[d[0]] == labels[d[1]] for d in ml]
        resultado_cl = [labels[d[0]] != labels[d[1]] for d in cl]    
        fase2_ml_atendidas = len(list(filter(lambda flag: flag == True, resultado_ml)))
        fase2_cl_atendidas = len(list(filter(lambda flag: flag == True, resultado_cl)))
        fase2_ml_falhas = len(list(filter(lambda flag: flag == False, resultado_ml)))
        fase2_cl_falhas = len(list(filter(lambda flag: flag == False, resultado_cl)))
    else:
        labels = 0
        score = 0
        score_sample = 0
        df['fase2_pckmeans_score'] = score
        df['fase2_pckmeans_score_sample'] = score_sample
        df['fase2_pckmeans_cluster'] = labels
        fase2_ml_atendidas = 0
        fase2_cl_atendidas = 0
        fase2_ml_falhas = 0
        fase2_cl_falhas = 0

    # Salvar base de dados para preservar processamento
    dfb = df.copy() # salva base selecionada

    # FASE 3: Agrupamento com Restrições K-means + customização de controle dos pares ML/CL, realocando para em caso de violação
    # Restaurar base para o processamento do K-means com as restrições criadas
    # Selecionando 3 variáveis para o agrupamento: classe_maturidade, IDHM, PIB_minmax
    X = df[['classe_maturidade','IDHM','PIBperCapita']].values

    # Processar o agrupamento com restrições utilizando o K-means adaptado
    kmeans = Constrained_KMeans(n_clusters=n_clusters, random_state=42, user_cluster_for_relocation=n_clusters+1)
    kmeans.fit(X, ml, cl)
    labels = kmeans.labels_
    score = silhouette_score(X, labels)
    score_sample = silhouette_samples(X, labels)
    df['fase3_c_kmeans_score'] = score
    df['fase3_c_kmeans_score_sample'] = score_sample
    df['fase3_c_kmeans_cluster'] = labels
    ref_ml = len(ml)
    ref_cl = len(cl)
    resultado_ml = [labels[d[0]] == labels[d[1]] for d in ml]
    resultado_cl = [labels[d[0]] != labels[d[1]] for d in cl]    
    fase3_ml_atendidas = len(list(filter(lambda flag: flag == True, resultado_ml)))
    fase3_cl_atendidas = len(list(filter(lambda flag: flag == True, resultado_cl)))
    fase3_ml_falhas = len(list(filter(lambda flag: flag == False, resultado_ml)))
    fase3_cl_falhas = len(list(filter(lambda flag: flag == False, resultado_cl)))

    # FASE 4: Agrupamento com Restrições K-means + customização de controle dos pares ML/CL com realocação
    # Gera lista final de municipios similares em ordem de aderência (maturidade, PIB e IDHM) e Restrições ML/CL
    # 
    lf = df[['id_municipio','outlier',
                'fase3_c_kmeans_cluster','fase3_c_kmeans_score_sample','fase3_c_kmeans_score',
                'fase2_pckmeans_cluster','fase2_pckmeans_score_sample','fase2_pckmeans_score',
                'fase1_c_kmeans_cluster','fase1_c_kmeans_score_sample','fase1_c_kmeans_score',
                'nome_municipio','sigla_uf','codigo_porte','populacao','Latitude','Longitude','distancia', 'nivel_maturidade','IDHM','PIBperCapita','PIBperCapita_salvo',
                'classe_maturidade','classe_IDHM','classe_PIB','adesaoRedeGOVBR']].sort_values(by=['fase3_c_kmeans_cluster','classe_maturidade','classe_PIB','classe_IDHM'],ascending=False)
    ref_cluster = lf[lf.id_municipio == ref_cidade]['fase3_c_kmeans_cluster'].iloc[0]
    df = lf[(lf['fase3_c_kmeans_cluster'] == ref_cluster)]
    dfalt = lf[(lf['fase3_c_kmeans_cluster'] == n_clusters+1)]    # Selecionar municipios realocados no cluster k+1
    ref_amostra_cluster_final       = len(df)
    ref_amostra_cluster_alternativo = len(lf[(lf['fase3_c_kmeans_cluster'] == n_clusters+1)])
    ref_amostra_cluster_outros      = len(lf[(lf['fase3_c_kmeans_cluster'] != ref_cluster) & (lf['fase3_c_kmeans_cluster'] != 5)])

    print("id: ", ref_cidade,"- cluster final:", ref_amostra_cluster_final,"- cluster alternativo:", ref_amostra_cluster_alternativo,"- cluster outros:", ref_amostra_cluster_outros)

    # Verificar se amostra resultante da Fase 3 é suficiente para lista final
    if ref_amostra_cluster_final > 2:
        # Amostra suficiente para lista final - mais de 2 municipios.
        df = lf[(lf['fase3_c_kmeans_cluster'] == ref_cluster)]  # Selecionar apenas municipios no mesmo cluster referência da Fase 3
        df['usou_cluster_alt'] = 0       # cluster criado suficiente, sem desvios, não usou o cluster da realocação.
        print("id: ", ref_cidade,"- Amostra suficiente para lista final - mais de 2 municipios")
    else:
        # Amostra insuficiente para lista final - até de 2 municipios - desvios. Aproveitar os municipios realocados e organizar no cluster, marcando como desvio
        df = lf[(lf['fase3_c_kmeans_cluster'] == ref_cluster) | (lf['fase3_c_kmeans_cluster'] == n_clusters+1)]  # Selecionar municipios no mesmo cluster da referência ou do cluster alternativo da Fase 3
        df['usou_cluster_alt'] = 1       # cluster criado aproveitando cluster alternativo da realocação, tem desvios.
        print("id: ", ref_cidade,"- Amostra insuficiente para lista final - até de 2 municipios - usou_cluster_alt")

    if ref_amostra_cluster_final in [2,3,4]:
        print("id: ", ref_cidade,"- Amostra suficiente para lista final, mas pequena - mais de 2 municipios e menos que 5. ****")

    # FASE: Organização do Cluster Referência
    # Organizar o agrupamento final e confirmar o Silhouette Score
    # Selecionar as 3 variáveis para o agrupamento: classe_maturidade, IDHM, PIB_minmax
    X = df[['classe_maturidade','IDHM','PIBperCapita']].values
    n_clusters = 2    
    ml = []
    cl = []
    kmeans = Constrained_KMeans(n_clusters=n_clusters, random_state=42, user_cluster_for_relocation=n_clusters+1)
    kmeans.fit(X, ml, cl)
    labels = kmeans.labels_
    score_final = silhouette_score(X,labels)
    score_sample_final = silhouette_samples(X, labels)
    df['cluster_final'] = labels
    df['score_sample_final'] = score_sample_final
    df['ref_score'] = score_final     # adicionar nova coluna com score
    df['ref_cidade'] = int(ref_cidade)     # adicionar nova coluna com ref_cidade
    df['ref_maturidade'] = ref_maturidade     # adicionar nova coluna com ref_cidade
    df['ref_amostra_selecao'] = ref_amostra_selecao     # adicionar nova coluna com total de amostras desta referência
    df['ref_amostra_cluster_inicial'] = ref_amostra_cluster_inicial     # adicionar nova coluna com total de amostras no cluster pckmeans desta referência
    df['ref_amostra_cluster_final'] = ref_amostra_cluster_final         # adicionar nova coluna com total de amostras no cluster final desta referência
    df['ref_amostra_cluster_alt'] = ref_amostra_cluster_alternativo     # adicionar nova coluna com total de amostras no cluster realocado
    df['ref_amostra_cluster_outros'] = ref_amostra_cluster_outros       # adicionar nova coluna com total de amostras em outros clusters
    df['ref_ml'] = ref_ml     # adicionar nova coluna com total de pares ML
    df['ref_cl'] = ref_cl     # adicionar nova coluna com total de pares CL
    df['fase3_ml_atendidas'] = fase3_ml_atendidas     # adicionar nova coluna com total de pares ML atendidos
    df['fase3_cl_atendidas'] = fase3_cl_atendidas     # adicionar nova coluna com total de pares CL atendidos 
    df['fase2_ml_atendidas'] = fase2_ml_atendidas     # adicionar nova coluna com total de pares ML atendidos
    df['fase2_cl_atendidas'] = fase2_cl_atendidas     # adicionar nova coluna com total de pares CL atendidos 
    df['fase2_ml_atendidas'] = fase2_ml_atendidas     # adicionar nova coluna com total de pares ML atendidos
    df['fase3_ml_falhas'] = fase3_ml_falhas     # adicionar nova coluna com total de pares ML violados
    df['fase3_cl_falhas'] = fase3_cl_falhas     # adicionar nova coluna com total de pares ML violados
    df['fase2_ml_falhas'] = fase2_ml_falhas     # adicionar nova coluna com total de pares ML violados
    df['fase2_cl_falhas'] = fase2_cl_falhas     # adicionar nova coluna com total de pares ML violados

    # FASE: Organização do Cluster Referência
    # Gera lista final de municipios similares em ordem de aderência (maturidade, PIB e IDHM) e Restrições ML/CL
    lf = df[['ref_cidade','id_municipio','ref_maturidade','outlier',
             'cluster_final','ref_score','score_sample_final','usou_cluster_alt',
                'fase3_c_kmeans_cluster','fase3_c_kmeans_score_sample','fase3_c_kmeans_score',
                'fase2_pckmeans_cluster','fase2_pckmeans_score_sample','fase2_pckmeans_score',
                'fase1_c_kmeans_cluster','fase1_c_kmeans_score_sample','fase1_c_kmeans_score',
                'ref_amostra_selecao','ref_amostra_cluster_inicial','ref_amostra_cluster_final','ref_amostra_cluster_alt','ref_amostra_cluster_outros',
                'ref_ml','ref_cl',
                'fase3_ml_atendidas','fase3_cl_atendidas','fase3_ml_falhas','fase3_cl_falhas',
                'fase2_ml_atendidas','fase2_cl_atendidas','fase2_ml_falhas','fase2_cl_falhas',
                'nome_municipio','sigla_uf','codigo_porte','populacao','Latitude','Longitude','distancia', 'nivel_maturidade','IDHM','PIBperCapita','PIBperCapita_salvo',
                'classe_maturidade','classe_IDHM','classe_PIB','adesaoRedeGOVBR']].sort_values(by=['cluster_final','score_sample_final','classe_maturidade','classe_PIB','classe_IDHM'],ascending=False)
    ref_cluster:DataFrame = lf[lf.id_municipio == ref_cidade]['cluster_final'].iloc[0]                         # Salvar novo cluster referência
    lf_ref:DataFrame = lf[(lf['id_municipio'] == ref_cidade)]                                                  # Salvar municipio referencia na lista final - controle 0 - apenas para o mapa e estatisticas na visualização
    lf_principal:DataFrame = lf[(lf['cluster_final'] == ref_cluster) & (lf['id_municipio'] != ref_cidade)]     # Retirar municipio referencia da lista final - controle 1
    lf_outros:DataFrame = lf[(lf['cluster_final'] != ref_cluster) & (lf['id_municipio'] != ref_cidade)]        # Retirar municipio referencia da lista final - controle 2

    # Adicionar coluna para controlar a visualização nas listas e nos mapas do Power BI
    lf_principal['controle'] = 1   
    lf_outros['controle'] = 2
    lf_ref['controle'] = 0

    # Em caso de agrupamento com outlier, restaurar valor original do PIB
    if outlier:
        print("d. id: ", ref_cidade,"outlier", outlier, "- valor pib antes da restauração: ", lf_ref[lf_ref.id_municipio == ref_cidade]['PIBperCapita'])
        lf_ref.loc[lf_ref.id_municipio == ref_cidade,'PIBperCapita'] = ref_pib
        print("e. id: ", ref_cidade,"outlier", outlier, "- valor pib antes da restauração: ", lf_ref[lf_ref.id_municipio == ref_cidade]['PIBperCapita'])

    # Mesclar linhas após estabelecer controle e criar ordenação dos municipios
    apresenta_municipios = pd.concat([lf_principal, lf_outros])
    apresenta_municipios.reset_index(inplace=True, drop=True)         # reinicializar indice após a seleção final - utilizado para o corte de 5.
    apresenta_municipios = apresenta_municipios.reset_index()
    apresenta_municipios = apresenta_municipios.head(5)     # Manter apenas o primeiros 5 registros
    apresenta_municipios = pd.concat([apresenta_municipios, lf_ref])  # adiciona referencia
    apresenta_municipios.reset_index(inplace=True, drop=True)         # reinicializar indice após a seleção final (será utilizado na ordem de visualização)
    apresenta_municipios.index.name = 'ordem'
    apresenta_municipios = apresenta_municipios.reset_index()

    # Restaurar PIBperCapita devido a erro no controle de outlier - investigar posteriormente
    apresenta_municipios = apresenta_municipios.rename(columns={'PIBperCapita': 'PIBperCapita_old'})
    apresenta_municipios = apresenta_municipios.rename(columns={'PIBperCapita_salvo': 'PIBperCapita'})

    # Devolver a lista final criada
    return(apresenta_municipios)


# Processar Agrupamento de todos os municipios
# Criar dicionário com as cidades usando id_municipio
cidades_lista = df.id_municipio
dft = pd.DataFrame()

# Exemplos de cidades para executar o notebook - depois poderá ser substituido pelo código de interação com o usuário (exemplo acima)
#lstMunicipios = [4319307]  # São Paulo das Missões/RS - porte 2
#lstMunicipios = [3106200]  # Belo Horizonte/MG - capital, porte 10
#lstMunicipios = [3550308]  # São Paulo/SP - capital, porte 10
#lstMunicipios = [3303807]  # Paraty/RJ - porte 4
#lstMunicipios = [3523008]  # Itapura/SP - porte 4
#lstMunicipios = [1702109]  # Araguaína/TO - porte 6
#lstMunicipios = [1302603]  # Manaus/AM - porte 10
#lstMunicipios = [3304557]  # Rio de Janeiro
#lstMunicipios = [1100122]  # Ji-Paraná
#lstMunicipios = [2609006]    ### Macaparana
#lstMunicipios = [1502152]   # OUTLIER - Canaã dos Carajás/PA - porte 5
#lstMunicipios = [3303302]   # OUTLIER - Niteroi - porte 5
#lstMunicipios = [2607208]   # OUTLIER - Ipojuca - porte 5
#lstMunicipios = [3505708]   # OUTLIER - Barueri - porte 5
#lstMunicipios = [1508357]   # OUTLIER - Vitória do Xingu	PA
#lstMunicipios = [5219605,5219704,5219712,5219738,5219753,5219803,5219902,5220009,5220058,5220108,5220157,5220207,5220264]
#lstMunicipios = [4315602]  # Rio Grande	RS
#lstMunicipios = [5211909]  # Jataí	GO
#lstMunicipios = [1400100]  # Boa Vista	RR	
#lstMunicipios = [1200401]  # Rio Branco	AC	  
#lstMunicipios = [3103009]  # Antônio Dias	MG	
#lstMunicipios = [2301000]  # Aquiraz	CE
#lstMunicipios = [2304285]  # Eusébio	CE
#lstMunicipios = [5106190]  # Nova Santa Helena	MT
#lstMunicipios = [5106505]  # Poconé	MT
#lstMunicipios = [5107925]  # Sorriso	MT	
#lstMunicipios = [5300108]  # Brasília  # 
#lstMunicipios = [2605459]  # Fernando de Noronha  # 
#lstMunicipios = [2905701]  # Camaçari	BA	-- uso do cluster alternativo
#lstMunicipios = [2602902]  # Cabo de Santo Agostinho	PE	-- uso do cluster alternativo
#lstMunicipios = [4304606]  ## Canoas	RS 
#lstMunicipios = [4305108]  ## Caxias do Sul	RS	
#lstMunicipios = [3520509]  ## Indaiatuba	SP	
#lstMunicipios = [5102504]  ## Cáceres	MT	
#lstMunicipios = [4125506]  ## Cáceres	MT	
#lstMunicipios = [2602902]  ## Cáceres	MT	
#lstMunicipios = [5103403]  ## Cáceres	MT	
#lstMunicipios = [5107909]  ## Cáceres	MT	
#lstMunicipios = [5100250]  ## Cáceres	MT	
#lstMunicipios = [5103403]  ## Cuiabá	MT	
#lstMunicipios = [5008305]  ## Três Lagoas	MS	
lstMunicipios = [3548906]  # São Carlos/SP - porte 7

execucao_producao = True
execucao_producao = False    # comentar para ativar PRODUCAO e executar a pesquisa para todos os municípios

import sys

try:
    if execucao_producao:
        # Chamar processo para cada municipio PRODUCAO
       for i in range(len(cidades_lista)):
            print("i:", i, "id:", cidades_lista[i])
            dfm = Processa_Municipio(cidades_lista[i], df, pib_vlr_zscore2)
            dft = pd.concat([dft, dfm])
    else:
        # Chamar processo para cada municipio TESTE
        for i in range(len(lstMunicipios)):
            print("i:", i, "id:", lstMunicipios[i])
            dfm = Processa_Municipio(lstMunicipios[i], df, pib_vlr_zscore2)
            dft = pd.concat([dft, dfm])
except Exception as err:
    if execucao_producao:
        print("**** erro produção - referência i:", i, "id:", cidades_lista[i])
        print(f"**** erro produção: {err=}, {type(err)=}")
    else:
        print("**** erro teste - referência i:", lstMunicipios[i])
        print(f"**** erro teste: {err=}, {type(err)=}")
    raise

# Gerar arquivo csv.    
dft.reset_index(inplace=True, drop=True)
dft.to_csv(caminhoDrive+arquivo_saida, sep=';', index=True)

i: 0 id: 3548906
id:  3548906 - amostra sem Brasilia e Fernando de Noronha: 151
id:  3548906 - cluster final: 32 - cluster alternativo: 37 - cluster outros: 82
id:  3548906 - Amostra suficiente para lista final - mais de 2 municipios


In [3]:
dft

,ordem,index,ref_cidade,id_municipio,ref_maturidade,outlier,cluster_final,ref_score,score_sample_final,usou_cluster_alt,...,distancia,nivel_maturidade,IDHM,PIBperCapita_old,PIBperCapita,classe_maturidade,classe_IDHM,classe_PIB,adesaoRedeGOVBR,controle
0,0,0.0,3548906,4115200,1,0.430438,0,0.671366,0.806533,0,...,443.850385,intermediário,0.808,51908.79,51908.79,2,4,3,1,1
1,1,1.0,3548906,3167202,1,0.430105,0,0.671366,0.806508,0,...,473.728312,avançado,0.760,51894.81,51894.81,4,4,3,1,1
2,2,2.0,3548906,3522505,1,0.448273,0,0.671366,0.805567,0,...,195.811017,aprimorado,0.735,52656.14,52656.14,3,3,3,1,1
3,3,3.0,3548906,3304557,1,0.458345,0,0.671366,0.803671,0,...,492.324926,avançado,0.799,53078.23,53078.23,4,4,3,1,1
4,4,4.0,3548906,3515004,1,0.414929,0,0.671366,0.803561,0,...,208.739263,intermediário,0.735,51258.86,51258.86,2,3,3,1,1
5,5,NaN,3548906,3548906,1,0.505277,0,0.671366,0.784156,0,...,0.000000,inicial,0.805,55044.88,55044.88,1,4,3,1,0
